In [1]:
import pandas as pd
import os
import json
import ast
from utils import *

In [2]:
pairs = pd.read_csv("trait_condition_pairs.csv")

In [3]:
all_traits = pd.read_csv("all_traits.csv")["Trait"].tolist()
all_traits = [normalize_trait(at) for at in all_traits]

In [4]:
rel = pd.read_csv("trait_related_genes.csv")
rel['Related_Genes'] = rel['Related_Genes'].apply(ast.literal_eval)
t2g = pd.Series(rel['Related_Genes'].values, index=rel['Trait']).to_dict()  # the mapping from trait to genes

In [5]:
gene_info_path = './trait_related_genes.csv'
data_root = '/home/techt/Desktop/a4s/gold_subset'
output_root = './output_lasso_tuned'

condition = None

for trait in all_traits:
    print(f"Trait {trait} only")
    output_dir = os.path.join(output_root, trait)
    os.makedirs(output_dir, exist_ok=True)
    try:
        trait_data, _, _ = select_and_load_cohort(data_root, trait, is_two_step=False)
        trait_data = trait_data.drop(columns=['Age', 'Gender'], errors="ignore")

        Y = trait_data[trait].values
        Z = None
        X = trait_data.drop(columns=[trait]).values

        has_batch_effect = detect_batch_effect(X)
        if False and has_batch_effect:
            model_constructor = VariableSelection
            fixed_params = {}
            tune_params = {'lamda': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10]}
        else:
            model_constructor = Lasso
            fixed_params = {}
            tune_params = {'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10]}
            # tune_params = {'alpha': [1]}

        best_config, best_performance = tune_hyperparameters(model_constructor, fixed_params, tune_params, X, Y, trait_data.columns, trait, gene_info_path, condition, Z)
        model = ResidualizationRegressor(model_constructor, best_config)
        normalized_X, _ = normalize_data(X)
        model.fit(normalized_X, Y)

        var_names = trait_data.columns.tolist()
        significant_genes = interpret_result(model, var_names, trait, condition)
        save_result(condition, significant_genes, best_performance, output_dir)

    except:
        print(f"Error processing trait {trait}")
        continue

Trait Breast_Cancer only


/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.524e-02, tolerance: 7.839e-03
  model = cd_fast.enet_coordinate_descent(
/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.602e-02, tolerance: 8.410e-03
  model = cd_fast.enet_coordinate_descent(
/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing reg

The cross-validation performance: {'prediction': {'accuracy': 0.9917695473251028, 'precision': 0.9972682766281318, 'recall': 0.9936696491855969, 'f1': 0.9954503676232271}, 'selection': {'precision': 0.22404000141776131, 'precision_at_50': 0.30400000000000005, 'recall': 0.12350492880613362, 'f1': 0.15922949975922202, 'jaccard': 0.08651366715826689, 'jaccard2': 0.06863155955943394}}


/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.075e-01, tolerance: 7.839e-03
  model = cd_fast.enet_coordinate_descent(
/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.134e-01, tolerance: 8.410e-03
  model = cd_fast.enet_coordinate_descent(
/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing reg

The cross-validation performance: {'prediction': {'accuracy': 0.9925925925925926, 'precision': 0.998189452124936, 'recall': 0.9937197274328582, 'f1': 0.9959354689360413}, 'selection': {'precision': 0.18565214405654745, 'precision_at_50': 0.34400000000000003, 'recall': 0.049682365826944144, 'f1': 0.07838610971076596, 'jaccard': 0.04079290895374419, 'jaccard2': 0.028205333011811218}}


...

/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.822e-03, tolerance: 7.730e-04
  model = cd_fast.enet_coordinate_descent(
/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.587e-03, tolerance: 6.811e-04
  model = cd_fast.enet_coordinate_descent(
/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing reg

The cross-validation performance: {'prediction': {'accuracy': 0.8444444444444443, 'precision': 0.8825396825396826, 'recall': 0.9083333333333334, 'f1': 0.8928506787330317}, 'selection': {'precision': 0.02248100918689154, 'precision_at_50': 0.024, 'recall': 0.01276595744680851, 'f1': 0.016254189301437502, 'jaccard': 0.008202274778328657, 'jaccard2': 0.011388828227899063}}
The cross-validation performance: {'prediction': {'accuracy': 0.8666666666666666, 'precision': 0.8825396825396826, 'recall': 0.9333333333333333, 'f1': 0.9061840120663651}, 'selection': {'precision': 0.024096638655462185, 'precision_at_50': 0.024096638655462185, 'recall': 0.00851063829787234, 'f1': 0.012574981543004798, 'jaccard': 0.0063631096012192025, 'jaccard2': 0.0}}
The cross-validation performance: {'prediction': {'accuracy': 0.888888888888889, 'precision': 0.8880952380952382, 'recall': 0.9666666666666668, 'f1': 0.9227106227106228}, 'selection': {'precision': 0.07063492063492063, 'precision_at_50': 0.07063492063492

/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.660e-02, tolerance: 6.839e-03
  model = cd_fast.enet_coordinate_descent(


The gene info file does not contain genes related to the feature 'Type_1_Diabetes_Risk'.
Error processing trait Type_1_Diabetes_Risk
Trait Sjögrens_Syndrome only
Error processing trait Sjögrens_Syndrome
Trait Tourette_Syndrome only
Error processing trait Tourette_Syndrome
Trait Underweight only
Error processing trait Underweight
Trait Psoriatic_Arthritis only
Error processing trait Psoriatic_Arthritis
Trait Social_Behavior only
Error processing trait Social_Behavior
Trait Susceptibility_to_Infections only
Error processing trait Susceptibility_to_Infections
Trait Obstructive_sleep_apnea only
Error processing trait Obstructive_sleep_apnea
Trait Pheochromocytoma_and_Paraganglioma only
Error processing trait Pheochromocytoma_and_Paraganglioma
Trait Uterine_Carcinosarcoma only
The gene info file does not contain genes related to the feature 'Uterine_Carcinosarcoma'.
Error processing trait Uterine_Carcinosarcoma
Trait Stroke only
Error processing trait Stroke
Trait Tuberous_Sclerosis only
Er

/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.244e-03, tolerance: 8.432e-04
  model = cd_fast.enet_coordinate_descent(


In [6]:
gene_info_path = './trait_related_genes.csv'
data_root = '/home/techt/Desktop/a4s/gold_subset'
output_root = './output_lasso_tuned'

for i, (index, row) in enumerate(pairs.iterrows()):
    try:
        trait, condition = row['Trait'], row['Condition']
        output_dir = os.path.join(output_root, trait)
        os.makedirs(output_dir, exist_ok=True)

        if condition in ['Age', 'Gender']:
            trait_data, _, _ = select_and_load_cohort(data_root, trait, condition, is_two_step=False)
            redundant_col = 'Age' if condition == 'Gender' else 'Gender'
            if redundant_col in trait_data.columns:
                trait_data = trait_data.drop(columns=[redundant_col])

            Y = trait_data[trait].values
            Z = trait_data[condition].values
            X = trait_data.drop(columns=[trait, condition]).values

            has_batch_effect = detect_batch_effect(X)
            if False and has_batch_effect:
                model_constructor = VariableSelection
                fixed_params = {}
                tune_params = {'lamda': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10]}
            else:
                model_constructor = Lasso
                fixed_params = {}
                tune_params = {'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10]}
                # tune_params = {'alpha': [1]}

            best_config, best_performance = tune_hyperparameters(model_constructor, fixed_params, tune_params, X, Y, trait_data.columns, trait, gene_info_path, condition, Z)

            model = ResidualizationRegressor(model_constructor, best_config)
            normalized_X, _ = normalize_data(X)
            normalized_Z, _ = normalize_data(Z)
            model.fit(normalized_X, Y, normalized_Z)

            var_names = trait_data.columns.tolist()
            interpret_result(model, var_names, trait, condition)
            save_result(condition, significant_genes, best_performance, output_dir)

        else:
            trait_data, condition_data, regressors = select_and_load_cohort(data_root, trait, condition, is_two_step=True, gene_info_path=gene_info_path)
            if regressors is None:
                print(f'No gene regressors for trait {trait} and condition {condition}')
                continue

            print("Common gene regressors for condition and trait", regressors)
            X_condition = condition_data[regressors].values
            Y_condition = condition_data[condition].values

            condition_type = 'binary' if len(np.unique(Y_condition)) == 2 else 'continuous'

            if condition_type == 'binary':
                if X_condition.shape[1] > X_condition.shape[0]:
                    model = LogisticRegression(penalty='l1', solver='liblinear', random_state=42)
                else:
                    model = LogisticRegression()
            else:
                if X_condition.shape[1] > X_condition.shape[0]:
                    model = Lasso()
                else:
                    model = LinearRegression()

            normalized_X_condition, _ = normalize_data(X_condition)
            model.fit(normalized_X_condition, Y_condition)

            regressors_in_trait = trait_data[regressors].values
            normalized_regressors_in_trait, _ = normalize_data(regressors_in_trait)
            if condition_type == 'binary':
                predicted_condition = model.predict_proba(normalized_regressors_in_trait)[:, 1]
            else:
                predicted_condition = model.predict(normalized_regressors_in_trait)

            trait_data[condition] = predicted_condition
            trait_data = trait_data.drop(columns=regressors)
            trait_data = trait_data.drop(columns=['Age', 'Gender'], errors='ignore')

            Y = trait_data[trait].values
            Z = trait_data[condition].values
            X = trait_data.drop(columns=[trait, condition]).values

            has_batch_effect = detect_batch_effect(X)
            if False and has_batch_effect:
                model_constructor = VariableSelection
                fixed_params = {}
                tune_params = {'lamda': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10]}
            else:
                model_constructor = Lasso
                fixed_params = {}
                tune_params = {'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10]}
                # tune_params = {'alpha': [1]}

            best_config, best_performance = tune_hyperparameters(model_constructor, fixed_params, tune_params, X, Y, trait_data.columns, trait, gene_info_path, condition, Z)

            model = ResidualizationRegressor(model_constructor, best_config)
            normalized_X, _ = normalize_data(X)
            normalized_Z, _ = normalize_data(Z)
            model.fit(normalized_X, Y, normalized_Z)

            var_names = trait_data.columns.tolist()
            interpret_result(model, var_names, trait, condition)
            save_result(condition, significant_genes, best_performance, output_dir)
    except Exception as e:
        print(f"Error processing row {i}, for the trait '{trait}' and the condition '{condition}'\n: {e}")
        continue


/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.232e-02, tolerance: 7.994e-03
  model = cd_fast.enet_coordinate_descent(
/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.051e-02, tolerance: 7.102e-03
  model = cd_fast.enet_coordinate_descent(
/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing reg

The cross-validation performance: {'prediction': {'accuracy': 0.9890909090909092, 'precision': 0.9979899497487438, 'recall': 0.9898914556092938, 'f1': 0.993917487742649}, 'selection': {'precision': 0.06649858088036478, 'precision_at_50': 0.06, 'recall': 0.11985347985347983, 'f1': 0.0855363778167671, 'jaccard': 0.0446870376395883, 'jaccard2': 0.033000246357149335}}


/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.202e-01, tolerance: 7.994e-03
  model = cd_fast.enet_coordinate_descent(
/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.477e-02, tolerance: 7.102e-03
  model = cd_fast.enet_coordinate_descent(
/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing reg

The cross-validation performance: {'prediction': {'accuracy': 0.99, 'precision': 0.9979848738642708, 'recall': 0.9908718477661566, 'f1': 0.9944138853708303}, 'selection': {'precision': 0.06060566505762481, 'precision_at_50': 0.11599999999999999, 'recall': 0.05157509157509157, 'f1': 0.05572183081556067, 'jaccard': 0.02866312398093852, 'jaccard2': 0.027684737098461025}}


/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.143e-01, tolerance: 7.994e-03
  model = cd_fast.enet_coordinate_descent(
/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.113e-01, tolerance: 7.102e-03
  model = cd_fast.enet_coordinate_descent(
/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing reg

...

/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.944e-03, tolerance: 5.556e-04
  model = cd_fast.enet_coordinate_descent(
/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.136e-03, tolerance: 5.890e-04
  model = cd_fast.enet_coordinate_descent(
/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing reg

The cross-validation performance: {'prediction': {'accuracy': 0.7333333333333333, 'precision': 0.8, 'recall': 0.7833333333333333, 'f1': 0.7690476190476191}, 'selection': {'precision': 0.0, 'precision_at_50': 0.0, 'recall': 0.0, 'f1': 0.0, 'jaccard': 0.0, 'jaccard2': 0.0}}


/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.781e-03, tolerance: 5.556e-04
  model = cd_fast.enet_coordinate_descent(
/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.425e-03, tolerance: 5.890e-04
  model = cd_fast.enet_coordinate_descent(
/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing reg

The cross-validation performance: {'prediction': {'accuracy': 0.9, 'precision': 0.95, 'recall': 0.9, 'f1': 0.9142857142857143}, 'selection': {'precision': 0.0, 'precision_at_50': 0.0, 'recall': 0.0, 'f1': 0.0, 'jaccard': 0.0, 'jaccard2': 0.0}}
The cross-validation performance: {'prediction': {'accuracy': 0.8333333333333334, 'precision': 0.95, 'recall': 0.8, 'f1': 0.8228571428571427}, 'selection': {'precision': 0.0, 'precision_at_50': 0.0, 'recall': 0.0, 'f1': 0.0, 'jaccard': 0.0, 'jaccard2': 0.0}}
The cross-validation performance: {'prediction': {'accuracy': 0.8333333333333334, 'precision': 0.95, 'recall': 0.8, 'f1': 0.8228571428571427}, 'selection': {'precision': 0.0, 'precision_at_50': 0.0, 'recall': 0.0, 'f1': 0.0, 'jaccard': 0.0, 'jaccard2': 0.0}}
The cross-validation performance: {'prediction': {'accuracy': 0.6333333333333333, 'precision': 0.7166666666666667, 'recall': 0.6333333333333333, 'f1': 0.6323809523809524}, 'selection': {'precision': 0.0, 'precision_at_50': 0.0, 'recall': 

/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.156e-04, tolerance: 6.958e-04
  model = cd_fast.enet_coordinate_descent(
